In [1]:
import pandas as pd
import requests

In [2]:
admis = pd.read_csv('../Data Files/NSS_Hospital_Inpatient_Admissions.csv',
                    infer_datetime_format=True, parse_dates=[8,9],
                   dtype= {6:'str',12:'str',14:'str',15:'str'})
visits = pd.read_csv('../Data Files/NSS_PrimaryCare_Office_Visits.csv',
                     infer_datetime_format=True, parse_dates=[12],
                    dtype={8:'str',9:'str',10:'str',11:'str'})

C:\Users\pfbab\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3165: DtypeWarning: Columns (3,5,17) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [3]:
admis.iloc[:,6:8]

,Attending Provider NPI,Facility Provider NPI
0,1417170523,1306890389
1,1669599197,1902803315
2,1407880511,1902803315
3,1407008907,1427055839
4,1518045608,1992818256
...,...,...
6208,1720043417,1689772592
6209,1215968474,1427055839
6210,1861831836,1689608150
6211,1750544516,1689608150


In [4]:
visits.iloc[:,8:12]

,Attending Provider NPI,Facility Provider NPI,Service Provider NPI,Provider Group NPI
0,NaN,NaN,1487856779,1043427248
1,1528030996,1902803315,1528030996,1043427248
2,NaN,NaN,1477579365,1043427248
3,1821178088,1104842020,1821178088,1407058787
4,NaN,NaN,1609004191,1184722779
...,...,...,...,...
231368,NaN,NaN,1851738108,1184722779
231369,NaN,NaN,1215163035,1043427248
231370,NaN,NaN,1215163035,1043427248
231371,NaN,NaN,1144252396,1962790014


In [5]:
npi = admis.iloc[:,6].append((admis.iloc[:,7], visits.iloc[:,8],
                              visits.iloc[:,9],visits.iloc[:,10],
                              visits.iloc[:,11]), ignore_index=True)

In [6]:
npi = npi.drop_duplicates().reset_index().dropna().rename(columns={0:'NPI'}).drop(columns=['index'])
npi

,NPI
0,1417170523
1,1669599197
2,1407880511
3,1407008907
4,1518045608
...,...
5411,1336222397
5412,1336150572
5413,1417017435
5414,1790964864


In [7]:
npi.iloc[2,0]

'1407880511'

In [8]:
endpoint = 'https://npiregistry.cms.hhs.gov/api/?version=2.0'
params = {'number' : '1407880511'}

In [9]:
response = requests.get(endpoint , params = params)

In [10]:
response

<Response [200]>

In [11]:
res = response.json()
res

{'result_count': 1,
 'results': [{'enumeration_type': 'NPI-1',
   'number': 1407880511,
   'last_updated_epoch': 1578914062,
   'created_epoch': 1152576000,
   'basic': {'first_name': 'CAROLYN',
    'last_name': 'SACHS',
    'middle_name': 'JOY',
    'credential': 'MD',
    'sole_proprietor': 'NO',
    'gender': 'F',
    'enumeration_date': '2006-07-11',
    'last_updated': '2020-01-13',
    'status': 'A',
    'name': 'SACHS CAROLYN',
    'certification_date': '2020-01-13'},
   'other_names': [],
   'addresses': [{'country_code': 'US',
     'country_name': 'United States',
     'address_purpose': 'LOCATION',
     'address_type': 'DOM',
     'address_1': '757 WESTWOOD PLZ',
     'address_2': 'RM 14-19',
     'city': 'LOS ANGELES',
     'state': 'CA',
     'postal_code': '900953075',
     'telephone_number': '310-825-2111'},
    {'country_code': 'US',
     'country_name': 'United States',
     'address_purpose': 'MAILING',
     'address_type': 'DOM',
     'address_1': '5767 W CENTURY BLV

In [12]:
res.keys()

dict_keys(['result_count', 'results'])

In [13]:
res['results'][0]

{'enumeration_type': 'NPI-1',
 'number': 1407880511,
 'last_updated_epoch': 1578914062,
 'created_epoch': 1152576000,
 'basic': {'first_name': 'CAROLYN',
  'last_name': 'SACHS',
  'middle_name': 'JOY',
  'credential': 'MD',
  'sole_proprietor': 'NO',
  'gender': 'F',
  'enumeration_date': '2006-07-11',
  'last_updated': '2020-01-13',
  'status': 'A',
  'name': 'SACHS CAROLYN',
  'certification_date': '2020-01-13'},
 'other_names': [],
 'addresses': [{'country_code': 'US',
   'country_name': 'United States',
   'address_purpose': 'LOCATION',
   'address_type': 'DOM',
   'address_1': '757 WESTWOOD PLZ',
   'address_2': 'RM 14-19',
   'city': 'LOS ANGELES',
   'state': 'CA',
   'postal_code': '900953075',
   'telephone_number': '310-825-2111'},
  {'country_code': 'US',
   'country_name': 'United States',
   'address_purpose': 'MAILING',
   'address_type': 'DOM',
   'address_1': '5767 W CENTURY BLVD',
   'address_2': 'SUITE 400',
   'city': 'LOS ANGELES',
   'state': 'CA',
   'postal_code'

In [14]:
res['results'][0].keys()

dict_keys(['enumeration_type', 'number', 'last_updated_epoch', 'created_epoch', 'basic', 'other_names', 'addresses', 'taxonomies', 'identifiers'])

In [15]:
res['results'][0]['enumeration_type']

'NPI-1'

In [16]:
endpoint = 'https://npiregistry.cms.hhs.gov/api/?version=2.0'
enumeration = list()
entry = []
for x in range(0,len(npi)):
    params = {'number' : npi.iloc[x,0]}
    print(str(x) + str(params))
    response = requests.get(endpoint , params = params)
    try:
        res = response.json()
        enumeration.append(res['results'][0]['enumeration_type'])
        entry.append(res['results'][0])
    except:
        enumeration.append('NPI-0')
        entry.append('Error')

0{'number': '1417170523'}
1{'number': '1669599197'}
2{'number': '1407880511'}
3{'number': '1407008907'}
4{'number': '1518045608'}
5{'number': '1346556891'}
6{'number': '1316989247'}
7{'number': '1417995317'}
8{'number': '1396818134'}
9{'number': '1447495387'}
10{'number': '1528163003'}
11{'number': '1366604969'}
12{'number': '1427375757'}
13{'number': '1619073954'}
14{'number': '1972526721'}
15{'number': '1366847147'}
16{'number': '1750346474'}
17{'number': '1932127248'}
18{'number': '1346566221'}
19{'number': '1598993990'}
20{'number': '1720240526'}
21{'number': '1023149044'}
22{'number': '1508841602'}
23{'number': '1730127069'}
24{'number': '1053334292'}
25{'number': '1922323856'}
26{'number': '1700828704'}
27{'number': '1982865044'}
28{'number': '1902850498'}
29{'number': '1710270442'}
30{'number': '1427154533'}
31{'number': '1306902739'}
32{'number': '1518963032'}
33{'number': '1750425278'}
34{'number': '1427344951'}
35{'number': '1073566931'}
36{'number': '1679541775'}
37{'number'

297{'number': '1922328590'}
298{'number': '1588624522'}
299{'number': '1023036068'}
300{'number': '1023029162'}
301{'number': '1154683043'}
302{'number': '1144301722'}
303{'number': '1851554372'}
304{'number': '1215961628'}
305{'number': '1871579458'}
306{'number': '1013973783'}
307{'number': '1609140524'}
308{'number': '1619040219'}
309{'number': '1851374441'}
310{'number': '1215183066'}
311{'number': '1730408774'}
312{'number': '1669570271'}
313{'number': '1326006941'}
314{'number': '1316011265'}
315{'number': '1104087808'}
316{'number': '1780614362'}
317{'number': '1316160211'}
318{'number': '1386651867'}
319{'number': '1487688198'}
320{'number': '1558310557'}
321{'number': '1295977197'}
322{'number': '1285750943'}
323{'number': '1760631287'}
324{'number': '1790966877'}
325{'number': '1548440902'}
326{'number': '1275544876'}
327{'number': '1073547808'}
328{'number': '1518999069'}
329{'number': '1245477652'}
330{'number': '1053331199'}
331{'number': '1588779870'}
332{'number': '19524

590{'number': '1033352638'}
591{'number': '1396828901'}
592{'number': '1487635298'}
593{'number': '1679508923'}
594{'number': '1023268182'}
595{'number': '1831279991'}
596{'number': '1497785208'}
597{'number': '1154536845'}
598{'number': '1053352302'}
599{'number': '1083971501'}
600{'number': '1407831530'}
601{'number': '1124182084'}
602{'number': '1700832201'}
603{'number': '1417986811'}
604{'number': '1568660165'}
605{'number': '1417951997'}
606{'number': '1598993248'}
607{'number': '1184607921'}
608{'number': '1326136185'}
609{'number': '1508807462'}
610{'number': '1730198128'}
611{'number': '1063647865'}
612{'number': '1265588164'}
613{'number': '1437163334'}
614{'number': '1184915001'}
615{'number': '1780901231'}
616{'number': '1619136838'}
617{'number': '1457309775'}
618{'number': '1114981594'}
619{'number': '1730335191'}
620{'number': '1366646952'}
621{'number': '1073733861'}
622{'number': '1245551175'}
623{'number': '1295780260'}
624{'number': '1992140073'}
625{'number': '14270

883{'number': '1285659482'}
884{'number': '1376515023'}
885{'number': '1801236401'}
886{'number': '1851311930'}
887{'number': '1255351854'}
888{'number': '1972528305'}
889{'number': '1013114719'}
890{'number': '1619048659'}
891{'number': '1427175827'}
892{'number': '1033361282'}
893{'number': '1861620593'}
894{'number': '1336184670'}
895{'number': '1770810590'}
896{'number': '1326134859'}
897{'number': '1497827562'}
898{'number': '1932121498'}
899{'number': '1841272150'}
900{'number': '1831144583'}
901{'number': '1730316969'}
902{'number': '1750492146'}
903{'number': '1538222310'}
904{'number': '1700047362'}
905{'number': '1194800094'}
906{'number': '1750371142'}
907{'number': '1205271285'}
908{'number': '1184828444'}
909{'number': '1417101668'}
910{'number': '1205190659'}
911{'number': '1164565776'}
912{'number': '1376641258'}
913{'number': '1508943432'}
914{'number': '1356372791'}
915{'number': '1508828179'}
916{'number': '1790050979'}
917{'number': '1386625721'}
918{'number': '10533

1170{'number': '1649235300'}
1171{'number': '1023151735'}
1172{'number': '1851617641'}
1173{'number': '1902067325'}
1174{'number': '1497980932'}
1175{'number': '1902833080'}
1176{'number': '1134232051'}
1177{'number': '1831175942'}
1178{'number': '1023245164'}
1179{'number': '1043379092'}
1180{'number': '1619311669'}
1181{'number': '1194160242'}
1182{'number': '1306000922'}
1183{'number': '1588898845'}
1184{'number': '1093767337'}
1185{'number': '1164450714'}
1186{'number': '1992913222'}
1187{'number': '1043338734'}
1188{'number': '1386683621'}
1189{'number': '1265574867'}
1190{'number': '1063402477'}
1191{'number': '1184682379'}
1192{'number': '1316967938'}
1193{'number': '1891956678'}
1194{'number': '1659442317'}
1195{'number': '1871631051'}
1196{'number': '1245566264'}
1197{'number': '1649505009'}
1198{'number': '1508165077'}
1199{'number': '1083654529'}
1200{'number': '1043218209'}
1201{'number': '1174565451'}
1202{'number': '1982939955'}
1203{'number': '1538170808'}
1204{'number':

...

4000{'number': '1821025479'}
4001{'number': '1467401455'}
4002{'number': '1831148725'}
4003{'number': '1396065215'}
4004{'number': '1548680143'}
4005{'number': '1558516476'}
4006{'number': '1497799589'}
4007{'number': '1134318918'}
4008{'number': '1295744076'}
4009{'number': '1649455841'}
4010{'number': '1295710390'}
4011{'number': '1356734917'}
4012{'number': '1033439898'}
4013{'number': '1477715613'}
4014{'number': '1710960307'}
4015{'number': '1003872870'}
4016{'number': '1710962592'}
4017{'number': '1285711937'}
4018{'number': '1578820395'}
4019{'number': '1861501926'}
4020{'number': '1295892149'}
4021{'number': '1477619823'}
4022{'number': '1801162227'}
4023{'number': '1366427353'}
4024{'number': '1942285309'}
4025{'number': '1861476483'}
4026{'number': '1619033057'}
4027{'number': '1326157421'}
4028{'number': '1205911062'}
4029{'number': '1235118233'}
4030{'number': '1598991788'}
4031{'number': '1861556391'}
4032{'number': '1780628487'}
4033{'number': '1932465879'}
4034{'number':

4283{'number': '1376806745'}
4284{'number': '1306064126'}
4285{'number': '1245211382'}
4286{'number': '1407844285'}
4287{'number': '1619130655'}
4288{'number': '1124053012'}
4289{'number': '1912963109'}
4290{'number': '1346350766'}
4291{'number': '1760562904'}
4292{'number': '1902127673'}
4293{'number': '1205037629'}
4294{'number': '1134306830'}
4295{'number': '1033281845'}
4296{'number': '1851430896'}
4297{'number': '1578677431'}
4298{'number': '1306894431'}
4299{'number': '1962422576'}
4300{'number': '1194892877'}
4301{'number': '1275961252'}
4302{'number': '1467485474'}
4303{'number': '1760409197'}
4304{'number': '1275793937'}
4305{'number': '1215919550'}
4306{'number': '1104841212'}
4307{'number': '1891184941'}
4308{'number': '1134258957'}
4309{'number': '1265521413'}
4310{'number': '1740222314'}
4311{'number': '1235143702'}
4312{'number': '1982790085'}
4313{'number': '1376852335'}
4314{'number': '1336175967'}
4315{'number': '1700941671'}
4316{'number': '1205129731'}
4317{'number':

4566{'number': '1396180758'}
4567{'number': '1851577241'}
4568{'number': '1689633844'}
4569{'number': '1578753141'}
4570{'number': '1508891748'}
4571{'number': '1104839778'}
4572{'number': '1568433274'}
4573{'number': '1356367791'}
4574{'number': '1336271204'}
4575{'number': '1619175676'}
4576{'number': '1629085964'}
4577{'number': '1396824199'}
4578{'number': '1225186620'}
4579{'number': '1679749865'}
4580{'number': '1205186442'}
4581{'number': '1700953122'}
4582{'number': '1700857703'}
4583{'number': '1104802354'}
4584{'number': '1750300067'}
4585{'number': '1639403884'}
4586{'number': '1851373310'}
4587{'number': '1477580892'}
4588{'number': '1053467274'}
4589{'number': '1396851523'}
4590{'number': '1467401521'}
4591{'number': '1578545836'}
4592{'number': '1831425388'}
4593{'number': '1407910847'}
4594{'number': '1922356807'}
4595{'number': '1386729358'}
4596{'number': '1992893945'}
4597{'number': '1275638876'}
4598{'number': '1609816107'}
4599{'number': '1437438538'}
4600{'number':

4849{'number': '1578852026'}
4850{'number': '1275962706'}
4851{'number': '1043238041'}
4852{'number': '1679685069'}
4853{'number': '1407922404'}
4854{'number': '1669548681'}
4855{'number': '1437113859'}
4856{'number': '1659312593'}
4857{'number': '1518993450'}
4858{'number': '1821049156'}
4859{'number': '1790027886'}
4860{'number': '1104206978'}
4861{'number': '1922258821'}
4862{'number': '1184939787'}
4863{'number': '1134328735'}
4864{'number': '1639190861'}
4865{'number': '1285645002'}
4866{'number': '1174632970'}
4867{'number': '1770843666'}
4868{'number': '1033182902'}
4869{'number': '1407872757'}
4870{'number': '1740238104'}
4871{'number': '1235118936'}
4872{'number': '1699940494'}
4873{'number': '1992134688'}
4874{'number': '1417928417'}
4875{'number': '1508870577'}
4876{'number': '1396744066'}
4877{'number': '1912096579'}
4878{'number': '1427443803'}
4879{'number': '1225265325'}
4880{'number': '1962448522'}
4881{'number': '1912011347'}
4882{'number': '1508821208'}
4883{'number':

5132{'number': '1538241252'}
5133{'number': '1235187311'}
5134{'number': '1245579655'}
5135{'number': '1124008727'}
5136{'number': '1518947167'}
5137{'number': '1205167350'}
5138{'number': '1477666352'}
5139{'number': '1720394711'}
5140{'number': '1992930986'}
5141{'number': '1174503767'}
5142{'number': '1386637346'}
5143{'number': '1972596989'}
5144{'number': '1831129816'}
5145{'number': '1770675886'}
5146{'number': '1699986950'}
5147{'number': '1215917810'}
5148{'number': '1144213166'}
5149{'number': '1942258033'}
5150{'number': '1154379394'}
5151{'number': '1902854078'}
5152{'number': '1407806904'}
5153{'number': '1578958450'}
5154{'number': '1003864174'}
5155{'number': '1154479723'}
5156{'number': '1760552343'}
5157{'number': '1457437154'}
5158{'number': '1447288345'}
5159{'number': '1265794929'}
5160{'number': '1770917478'}
5161{'number': '1356304034'}
5162{'number': '1033152350'}
5163{'number': '1366583932'}
5164{'number': '1306938055'}
5165{'number': '1619925153'}
5166{'number':

In [17]:
npidf = pd.DataFrame()
npidf['npi'] = npi.iloc[:,0]

In [18]:
npidf['enumeration'] = enumeration

In [19]:
npidf['entry'] = entry

In [20]:
npidf

,npi,enumeration,entry
0,1417170523,NPI-1,"{'enumeration_type': 'NPI-1', 'number': 141717..."
1,1669599197,NPI-1,"{'enumeration_type': 'NPI-1', 'number': 166959..."
2,1407880511,NPI-1,"{'enumeration_type': 'NPI-1', 'number': 140788..."
3,1407008907,NPI-1,"{'enumeration_type': 'NPI-1', 'number': 140700..."
4,1518045608,NPI-1,"{'enumeration_type': 'NPI-1', 'number': 151804..."
...,...,...,...
5411,1336222397,NPI-2,"{'enumeration_type': 'NPI-2', 'number': 133622..."
5412,1336150572,NPI-1,"{'enumeration_type': 'NPI-1', 'number': 133615..."
5413,1417017435,NPI-2,"{'enumeration_type': 'NPI-2', 'number': 141701..."
5414,1790964864,NPI-2,"{'enumeration_type': 'NPI-2', 'number': 179096..."


In [21]:
npidf.to_csv('../output/npi.csv')